In [1]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LeakyReLU, Input, Flatten, MaxPooling1D, Conv1D, MaxPool1D, GlobalMaxPool1D, Dense, AveragePooling1D, GlobalAveragePooling1D

In [2]:
import matplotlib.pyplot as plt
%matplotlib tk

In [3]:
import os

In [4]:
path = r"C:\Users\mata2\Desktop\master\podaci\0k\X"

In [5]:
dir = os.listdir(path)

Ucitavanje fajlova

In [6]:
file_names = []
for file in dir:
    file_names.append(file)

In [7]:
percent_train = 0.8

In [8]:
train_files_names = file_names[:int(percent_train*len(file_names))]
test_file_names = file_names[int(percent_train*len(file_names))::]

Ucitavanje zadate trajektorije

In [9]:
x_command = np.asarray(pd.read_csv(r"C:\Users\mata2\Desktop\master\podaci\xCmd.txt",header=None).astype('int'))

Konkatenacija fajlova koji se posle pravilno rasporedjuju na adekvatne train/test skupove

In [10]:
train_ = np.concatenate([pd.read_csv(path + "/" + file, header=None)[:61000].astype('int')
                              for file in train_files_names], axis=0)
test_ = np.concatenate([pd.read_csv(path + "/" + file, header=None)[:61000].astype('int')
                              for file in test_file_names], axis=0)

Prethodno konkatenizovani podaci se adekvatno dele kako bi se izbeglo fiktivno stanje (stanje pri kraju rada i pocetku novog ako su konkatenizovani podaci)

In [12]:
for j in range(len(train_files_names)):
    exec(f"razlika_trening_{j} = []")
    for i in range(61000):
        exec(f"razlika_trening_{j}.append(x_command[i] - train_[j*61000+i])")
    exec(f'razlika_trening_{j} = np.asarray(razlika_trening_{j})')        

In [11]:
for j in range(len(test_file_names)):
    exec(f"razlika_test_{j} = []")
    for i in range(61000):
        exec(f"razlika_test_{j}.append(x_command[i] - test_[j*61000+i])")
    exec(f'razlika_test_{j} = np.asarray(razlika_test_{j})')    

Funkcija koja deli podatke na "sliding window"

In [14]:
def create_timeseries_split(data, time_steps):

    # Get the number of samples in the data
    num_samples = len(data) - time_steps

    # Create empty arrays to store features and target values
    x_train = np.zeros((num_samples, time_steps, 1))
    y_train = np.zeros((num_samples, 1))

    # Loop through the data and create features and target values
    for i in range(num_samples):
        # Extract a slice of data for the current feature
        x_train[i] = data[i:i+time_steps, :]

        # The target value is the next value after the feature
        y_train[i] = data[i+time_steps, 0]

    return x_train, y_train

Velicina buffera

In [13]:
time_steps = 8

Deljenje podataka na train i test skup

In [15]:
for i in range(len(train_files_names)):
    exec(f'train_X_{i}, train_y_{i} = create_timeseries_split(razlika_trening_{i}, time_steps)')

for i in range(len(test_file_names)):
    exec(f'test_X_{i}, test_y_{i} = create_timeseries_split(razlika_test_{i}, time_steps)')

train_X = np.asarray(train_X_0)
for i in range(1,len(train_files_names)):
    exec(f'train_X = np.append(train_X, train_X_{i}, axis=0)')

train_y = np.asarray(train_y_0)
for i in range(1,len(train_files_names)):
    exec(f'train_y = np.append(train_y, train_y_{i}, axis=0)')

test_X = np.asarray(test_X_0)
for i in range(1,len(test_file_names)):
    exec(f'test_X = np.append(test_X, test_X_{i}, axis=0)')

test_y = np.asarray(test_y_0)
for i in range(1,len(test_file_names)):
    exec(f'test_y = np.append(test_y, test_y_{i}, axis=0)')

In [16]:
input_shape = (time_steps, 1)
input_shape

(8, 1)

In [33]:
keras.utils.set_random_seed(7)
optimizerMSE = keras.optimizers.Adam(learning_rate=0.0001)

modelMSE = Sequential()
modelMSE.add(Input(shape=input_shape))
modelMSE.add(Conv1D(filters=8, kernel_size=3,padding='same', activation=LeakyReLU()))
modelMSE.add(MaxPooling1D(pool_size=3))
modelMSE.add(Flatten())
modelMSE.add(Dense(8, activation=LeakyReLU()))
modelMSE.add(Dense(units = 1, activation=LeakyReLU()))
modelMSE.compile(loss='mean_squared_error', optimizer = optimizerMSE)

In [34]:
historyMSE = modelMSE.fit(train_X, train_y, epochs=100, batch_size = 256, shuffle=False, verbose = 2)

Epoch 1/100
5718/5718 - 10s - loss: 19.5658 - 10s/epoch - 2ms/step
Epoch 2/100
5718/5718 - 8s - loss: 12.5449 - 8s/epoch - 1ms/step
Epoch 3/100
5718/5718 - 8s - loss: 11.0034 - 8s/epoch - 1ms/step
Epoch 4/100
5718/5718 - 8s - loss: 10.2403 - 8s/epoch - 1ms/step
Epoch 5/100
5718/5718 - 8s - loss: 9.7164 - 8s/epoch - 1ms/step
Epoch 6/100
5718/5718 - 8s - loss: 9.2421 - 8s/epoch - 1ms/step
Epoch 7/100
5718/5718 - 8s - loss: 8.8336 - 8s/epoch - 1ms/step
Epoch 8/100
5718/5718 - 8s - loss: 8.5530 - 8s/epoch - 1ms/step
Epoch 9/100
5718/5718 - 8s - loss: 8.3746 - 8s/epoch - 1ms/step
Epoch 10/100
5718/5718 - 8s - loss: 8.2441 - 8s/epoch - 1ms/step
Epoch 11/100
5718/5718 - 8s - loss: 8.1341 - 8s/epoch - 1ms/step
Epoch 12/100
5718/5718 - 8s - loss: 8.0282 - 8s/epoch - 1ms/step
Epoch 13/100
5718/5718 - 8s - loss: 7.9153 - 8s/epoch - 1ms/step
Epoch 14/100
5718/5718 - 8s - loss: 7.7893 - 8s/epoch - 1ms/step
Epoch 15/100
5718/5718 - 8s - loss: 7.6614 - 8s/epoch - 1ms/step
Epoch 16/100
5718/5718 - 8s 

In [35]:
train_scores = modelMSE.evaluate(test_X, test_y)

11436/11436 [==============================] - 12s 1ms/step - loss: 6.1375


In [21]:
#train_predict_MSE = modelMSE.predict(train_X)
test_predict_MSE = modelMSE.predict(test_X)

11436/11436 [==============================] - 14s 1ms/step


In [20]:
modelMSE.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 8, 8)              32        
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 2, 8)              0         
 g1D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 16)                0         
                                                                 
 dense_2 (Dense)             (None, 8)                 136       
                                                                 
 dense_3 (Dense)             (None, 1)                 9         
                                                                 
Total params: 177 (708.00 Byte)
Trainable params: 177 (708.00 Byte)
Non-trainable params: 0 (0.00 Byte)
________________

Ucitavamo standardne devijacije i srednje vrednosti

In [19]:
std_dev = []
with open(r"C:\Users\mata2\Desktop\master\Axis-projection-RNN\odstupanja\0k\std_dev.txt", 'r') as lines:
    for line in lines:
        std_dev.append(float(line.strip('\n')))

In [20]:
sr_vrednost = []
with open(r"C:\Users\mata2\Desktop\master\Axis-projection-RNN\odstupanja\0k\sr_vrednost.txt", 'r') as lines:
    for line in lines:
        sr_vrednost.append(float(line.strip('\n')))

mean 3 sigma

In [23]:
i = time_steps
mean_3std = 0
for j in range(len(test_predict_MSE)):
    if i%len(x_command) == 0:
        i = time_steps
    if sr_vrednost[i] - 3*std_dev[i] <= test_predict_MSE[j] <= sr_vrednost[i] + 3*std_dev[i]:
            mean_3std += 1
    i += 1
print(mean_3std, mean_3std/len(test_predict_MSE))

361829 0.988733495103183


pravljenje min/max koord, posle se koriste za plot

In [17]:
min_kord = [float('inf') for x in range(61000)]
max_kord = [float('-inf') for x in range(61000)]

for i in range(len(x_command)):    
    for j in range(len(train_files_names)):
        exec(f'current_file = razlika_trening_{j}')
        if current_file[i][0] < min_kord[i]:
            min_kord[i] = current_file[i][0]
        if current_file[i][0] > max_kord[i]:
            max_kord[i] = current_file[i][0]

min max pojas ZA KOORDINATE

In [54]:
i = time_steps
uspeli_minmax = 0
for j in range(len(test_predict_MSE)):
    if i%len(x_command) == 0:
        i = time_steps
    if min_kord[i] <= (test_predict_MSE[j]) <= max_kord[i]:
        uspeli_minmax += 1
    i += 1
uspeli_minmax/len(test_predict_MSE)

0.8905293590416229

cuvamo/ucitavamo model

# NA TEST SKUPU - plotovi trajektorija

test_ = realizovana trajektorija

test_predict_MSE = predvidjena RAZLIKA koordinata od x_command

razlika_test_{} = realizovana RAZLIKA od test podataka od x_command - CELA TRAJEKTORIJA

test_y_{} = REALIZOVANA y koordinata od razlika_test_{}

X_COMMAND - TEST_ = ODSTUPANJE/RAZLIKA_TEST_

X_COMMAND[TIME_STEPS] + test_predict_MSE[0] = PRVU KOORDINATU TRAJEKTORIJE PREDVIDJENE VREDNOSTI

In [22]:
command_length = len(x_command)

In [23]:
for j in range(len(test_file_names)):
    exec(f'predvidjena_trajektorija_{j} = []')
    for i in range(61000 - time_steps):
        exec(f'predvidjena_trajektorija_{j}.append(x_command[i + time_steps] + test_predict_MSE[j*(61000-time_steps) + i])')

In [24]:
for j in range(len(test_file_names)):
    exec(f'realizovana_trajektorija_{j} = []')
    for i in range(61000 - time_steps):
        exec(f'realizovana_trajektorija_{j}.append(test_[j*61000 + i])')

In [71]:
plt.figure(figsize=(12,7))
plt.subplot(211)
plt.axvline(x = 4795, color = 'b', label = 'axvline - full height')
plt.plot(x_command[time_steps::],'r')
plt.plot(predvidjena_trajektorija_0,'b')
plt.plot(realizovana_trajektorija_0[time_steps::], 'g')
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Х осе машине (μm)', fontsize=12)
plt.show()

plt.subplot(212)
plt.axvline(x = 4795, color = 'b', label = 'axvline - full height')
plt.plot(x_command[time_steps::],'r')
plt.plot(predvidjena_trajektorija_0,'b')
plt.plot(realizovana_trajektorija_0[time_steps::], 'g')
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Х осе машине (μm)', fontsize=12)
plt.legend(["_HiddenLabel","задата трајекторија", "предвиђена трајекторија 1", "реализована трајекторија 1"])
plt.show()

C:\Users\mata2\AppData\Local\Temp\ipykernel_11548\1984467602.py:18: UserWarning: The label '_HiddenLabel' of <matplotlib.lines.Line2D object at 0x0000023EC5781ED0> starts with '_'. It is thus excluded from the legend.
  plt.legend(["_HiddenLabel","задата трајекторија", "предвиђена трајекторија 1", "реализована трајекторија 1"])


In [72]:
plt.figure(figsize=(12,7))
plt.subplot(211)
plt.axvline(x = 8400, color = 'b', label = 'axvline - full height')
plt.plot(x_command[time_steps::],'r')
plt.plot(predvidjena_trajektorija_1,'b')
plt.plot(realizovana_trajektorija_1[time_steps::], 'g')
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Х осе машине (μm)', fontsize=12)
plt.show()

plt.subplot(212)
plt.axvline(x = 8400, color = 'b', label = 'axvline - full height')
plt.plot(x_command[time_steps::],'r')
plt.plot(predvidjena_trajektorija_1,'b')
plt.plot(realizovana_trajektorija_1[time_steps::], 'g')
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Х осе машине (μm)', fontsize=12)
plt.legend(["_HiddenLabel","задата трајекторија", "предвиђена трајекторија 2", "реализована трајекторија 2"])
plt.show()

C:\Users\mata2\AppData\Local\Temp\ipykernel_11548\2067760657.py:18: UserWarning: The label '_HiddenLabel' of <matplotlib.lines.Line2D object at 0x0000023EC5974BD0> starts with '_'. It is thus excluded from the legend.
  plt.legend(["_HiddenLabel","задата трајекторија", "предвиђена трајекторија 2", "реализована трајекторија 2"])


In [73]:
plt.figure(figsize=(12,7))
plt.subplot(211)
plt.axvline(x = 17000, color = 'b', label = 'axvline - full height')
plt.plot(x_command[time_steps::],'r')
plt.plot(predvidjena_trajektorija_2,'b')
plt.plot(realizovana_trajektorija_2[time_steps::], 'g')
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Х осе машине (μm)', fontsize=12)
plt.show()

plt.subplot(212)
plt.axvline(x = 17000, color = 'b', label = 'axvline - full height')
plt.plot(x_command[time_steps::],'r')
plt.plot(predvidjena_trajektorija_2,'b')
plt.plot(realizovana_trajektorija_2[time_steps::], 'g')
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Х осе машине (μm)', fontsize=12)
plt.legend(["Вертикална линија на индексу 17000","задата трајекторија", "предвиђена трајекторија 3", "реализована трајекторија 3"])
plt.show()

In [74]:
plt.figure(figsize=(12,7))
plt.subplot(211)
plt.axvline(x = 24250, color = 'b', label = 'axvline - full height')
plt.plot(x_command[time_steps::],'r')
plt.plot(predvidjena_trajektorija_3,'b')
plt.plot(realizovana_trajektorija_3[time_steps::], 'g')
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Х осе машине (μm)', fontsize=12)
plt.show()

plt.subplot(212)
plt.axvline(x = 24250, color = 'b', label = 'axvline - full height')
plt.plot(x_command[time_steps::],'r')
plt.plot(predvidjena_trajektorija_3,'b')
plt.plot(realizovana_trajektorija_3[time_steps::], 'g')
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Х осе машине (μm)', fontsize=12)
plt.legend(["_HiddenLabel","задата трајекторија", "предвиђена трајекторија 4", "реализована трајекторија 4"])
plt.show()

C:\Users\mata2\AppData\Local\Temp\ipykernel_11548\2521727846.py:18: UserWarning: The label '_HiddenLabel' of <matplotlib.lines.Line2D object at 0x0000023ECC39CBD0> starts with '_'. It is thus excluded from the legend.
  plt.legend(["_HiddenLabel","задата трајекторија", "предвиђена трајекторија 4", "реализована трајекторија 4"])


In [75]:
plt.figure(figsize=(12,7))
plt.subplot(211)
plt.axvline(x = 32420, color = 'b', label = 'axvline - full height')
plt.plot(x_command[time_steps::],'r')
plt.plot(predvidjena_trajektorija_4,'b')
plt.plot(realizovana_trajektorija_4[time_steps::], 'g')
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Х осе машине (μm)', fontsize=12)
plt.show()

plt.subplot(212)
plt.axvline(x = 32420, color = 'b', label = 'axvline - full height')
plt.plot(x_command[time_steps::],'r')
plt.plot(predvidjena_trajektorija_4,'b')
plt.plot(realizovana_trajektorija_4[time_steps::], 'g')
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Х осе машине (μm)', fontsize=12)
plt.legend(["_HiddenLabel","задата трајекторија", "предвиђена трајекторија 5", "реализована трајекторија 5"])
plt.show()

C:\Users\mata2\AppData\Local\Temp\ipykernel_11548\1523716765.py:18: UserWarning: The label '_HiddenLabel' of <matplotlib.lines.Line2D object at 0x0000023EAC9C7F10> starts with '_'. It is thus excluded from the legend.
  plt.legend(["_HiddenLabel","задата трајекторија", "предвиђена трајекторија 5", "реализована трајекторија 5"])


In [76]:
plt.figure(figsize=(12,7))
plt.subplot(211)
plt.axvline(x = 36300, color = 'b', label = 'axvline - full height')
plt.plot(x_command[time_steps::],'r')
plt.plot(predvidjena_trajektorija_5,'b')
plt.plot(realizovana_trajektorija_5[time_steps::], 'g')
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Х осе машине (μm)', fontsize=12)
plt.show()

plt.subplot(212)
plt.axvline(x = 36300, color = 'b', label = 'axvline - full height')
plt.plot(x_command[time_steps::],'r')
plt.plot(predvidjena_trajektorija_5,'b')
plt.plot(realizovana_trajektorija_5[time_steps::], 'g')
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Х осе машине (μm)', fontsize=12)
plt.legend(["Вертикална линија на индексу 36300","задата трајекторија", "предвиђена трајекторија 6", "реализована трајекторија 6"])
plt.show()

# NA TEST SKUPU - min/max plotovi

MIN MAX TRAJEKTORIJE

In [48]:
min_trajektorija = []
max_trajektorija = []
for i in range(61000):
    min_trajektorija.append(x_command[i] + min_kord[i])
    max_trajektorija.append(x_command[i] + max_kord[i])

In [49]:
trajektorija_koord = 1
i = time_steps
mean_3std_koord = 0
for j in range(len(test_predict_MSE)):
    if i%len(x_command) == 0:
        print("trajektorija ",trajektorija_koord, " sadrzi ", mean_3std_koord , " tacaka", ", procenat tri sigma iznosi: ",round(mean_3std_koord/61000*100,2), "%")
        exec(f"trajektorija_koord_{trajektorija_koord} = mean_3std_koord")
        trajektorija_koord += 1
        mean_3std_koord = 0
        i = time_steps
    if min_kord[i] <= (test_predict_MSE[j]) <= max_kord[i]:
            mean_3std_koord += 1
    i += 1
trajektorija_koord_6 = mean_3std_koord
print("trajektorija ",trajektorija_koord, " sadrzi ", mean_3std_koord , " tacaka", ", procenat tri sigma iznosi: ",round(mean_3std_koord/61000*100,2),"%")

trajektorija  1  sadrzi  52829  tacaka , procenat tri sigma iznosi:  86.6 %
trajektorija  2  sadrzi  54386  tacaka , procenat tri sigma iznosi:  89.16 %
trajektorija  3  sadrzi  58091  tacaka , procenat tri sigma iznosi:  95.23 %
trajektorija  4  sadrzi  56040  tacaka , procenat tri sigma iznosi:  91.87 %
trajektorija  5  sadrzi  53268  tacaka , procenat tri sigma iznosi:  87.32 %
trajektorija  6  sadrzi  51277  tacaka , procenat tri sigma iznosi:  84.06 %


In [52]:
plt.figure(figsize=(11,7))
plt.xlabel('време (ms)', fontsize=18)
plt.ylabel('растојање Х осе машине (μm)', fontsize=18)
plt.plot(min_trajektorija, color = 'b')
plt.plot(max_trajektorija, color = 'r')
for i in range(len(test_file_names)):
    exec(f'plt.plot(realizovana_trajektorija_{i})')
plt.legend(['мин трајекторија', "макс трајекторија", "реализована трајекторија 1","реализована трајекторија 2","реализована трајекторија 3","реализована трајекторија 4","реализована трајекторија 5","реализована трајекторија 6"])
plt.show()

In [50]:
plt.figure(figsize=(11,7))
plt.xlabel('време (ms)', fontsize=18)
plt.ylabel('растојање Х осе машине (μm)', fontsize=18)
plt.plot(min_trajektorija, color = 'b')
plt.plot(max_trajektorija, color = 'r')
for i in range(len(test_file_names)):
    exec(f'plt.plot(predvidjena_trajektorija_{i})')
plt.legend(['мин трајекторија', "макс трајекторија", "предвиђена трајекторија 1","предвиђена трајекторија 2","предвиђена трајекторија 3","предвиђена трајекторија 4","предвиђена трајекторија 5","предвиђена трајекторија 6"])
plt.show()

In [38]:
i = time_steps
plt.figure(figsize=(11,7))
for j in range(len(predvidjena_trajektorija_0)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_0[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_0[j], color='black', marker='o', markersize=2)
    else:
        plt.plot(i, predvidjena_trajektorija_0[j], color='red',marker='o',markersize=2)
    i += 1
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Х осе машине (μm)', fontsize=12)
plt.title('Количина тачака која упада у мин макс појас износи: {}%'.format(round((trajektorija_koord_1/61000)*100,2)) , fontsize=18)
plt.show()

In [37]:
i = time_steps
plt.figure(figsize=(12,7))
plt.subplot(211)
for j in range(len(predvidjena_trajektorija_0)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_0[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_0[j], color='black', marker='o', markersize=4)
    else:
        plt.plot(i, predvidjena_trajektorija_0[j], color='red',marker='o',markersize=4)
    i += 1
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Х осе машине (μm)', fontsize=12)
plt.title('Количина тачака која упада у мин/макс појас износи: {}%'.format(round((trajektorija_koord_1/61000)*100,2)) , fontsize=18)
plt.axvline(x = 21150, color = 'b', label = 'axvline - full height')

i = time_steps
plt.subplot(212)
for j in range(len(predvidjena_trajektorija_0)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_0[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_0[j], color='black', marker='o', markersize=4)
    else:
        plt.plot(i, predvidjena_trajektorija_0[j], color='red',marker='o',markersize=4)
    i += 1
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Х осе машине (μm)', fontsize=12)
plt.axvline(x = 21150, color = 'b', label = 'axvline - full height')
plt.show()

In [108]:
i = time_steps
plt.figure(figsize=(12,7))
plt.subplot(211)
for j in range(len(predvidjena_trajektorija_1)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_1[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_1[j], color='black', marker='o', markersize=4)
    else:
        plt.plot(i, predvidjena_trajektorija_1[j], color='red',marker='o',markersize=4)
    i += 1
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Х осе машине (μm)', fontsize=12)
plt.title('Количина тачака која упада у мин/макс појас износи: {}%'.format(round((trajektorija_koord_2/61000)*100,2)) , fontsize=18)
plt.axvline(x = 36110, color = 'b', label = 'axvline - full height')

i = time_steps
plt.subplot(212)
for j in range(len(predvidjena_trajektorija_1)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_1[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_1[j], color='black', marker='o', markersize=4)
    else:
        plt.plot(i, predvidjena_trajektorija_1[j], color='red',marker='o',markersize=4)
    i += 1
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Х осе машине (μm)', fontsize=12)
plt.axvline(x = 36110, color = 'b', label = 'axvline - full height')
plt.show()

In [109]:
i = time_steps
plt.figure(figsize=(12,7))
plt.subplot(211)
for j in range(len(predvidjena_trajektorija_2)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_2[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_2[j], color='black', marker='o', markersize=4)
    else:
        plt.plot(i, predvidjena_trajektorija_2[j], color='red',marker='o',markersize=4)
    i += 1
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Х осе машине (μm)', fontsize=12)
plt.title('Количина тачака која упада у мин/макс појас износи: {}%'.format(round((trajektorija_koord_3/61000)*100,2)) , fontsize=18)
plt.axvline(x = 36110, color = 'b', label = 'axvline - full height')

i = time_steps
plt.subplot(212)
for j in range(len(predvidjena_trajektorija_2)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_2[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_2[j], color='black', marker='o', markersize=4)
    else:
        plt.plot(i, predvidjena_trajektorija_2[j], color='red',marker='o',markersize=4)
    i += 1
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Х осе машине (μm)', fontsize=12)
plt.axvline(x = 36110, color = 'b', label = 'axvline - full height')
plt.show()

In [110]:
i = time_steps
plt.figure(figsize=(12,7))
plt.subplot(211)
for j in range(len(predvidjena_trajektorija_3)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_3[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_3[j], color='black', marker='o', markersize=4)
    else:
        plt.plot(i, predvidjena_trajektorija_3[j], color='red',marker='o',markersize=4)
    i += 1
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Х осе машине (μm)', fontsize=12)
plt.title('Количина тачака која упада у мин/макс појас износи: {}%'.format(round((trajektorija_koord_4/61000)*100,2)) , fontsize=18)
plt.axvline(x = 38200, color = 'b', label = 'axvline - full height')

i = time_steps
plt.subplot(212)
for j in range(len(predvidjena_trajektorija_3)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_3[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_3[j], color='black', marker='o', markersize=4)
    else:
        plt.plot(i, predvidjena_trajektorija_3[j], color='red',marker='o',markersize=4)
    i += 1
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Х осе машине (μm)', fontsize=12)
plt.axvline(x = 38200, color = 'b', label = 'axvline - full height')
plt.show()

In [29]:
i = time_steps
plt.figure(figsize=(12,7))
plt.subplot(211)
for j in range(len(predvidjena_trajektorija_4)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_4[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_4[j], color='black', marker='o', markersize=4)
    else:
        plt.plot(i, predvidjena_trajektorija_4[j], color='red',marker='o',markersize=4)
    i += 1
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Х осе машине (μm)', fontsize=12)
plt.title('Количина тачака која упада у мин/макс појас износи: {}%'.format(round((trajektorija_koord_5/61000)*100,2)) , fontsize=18)
plt.axvline(x = 40000, color = 'b', label = 'axvline - full height')

i = time_steps
plt.subplot(212)
for j in range(len(predvidjena_trajektorija_4)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_4[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_4[j], color='black', marker='o', markersize=4)
    else:
        plt.plot(i, predvidjena_trajektorija_4[j], color='red',marker='o',markersize=4)
    i += 1
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Х осе машине (μm)', fontsize=12)
plt.axvline(x = 40000, color = 'b', label = 'axvline - full height')
plt.show()

In [30]:
i = time_steps
plt.figure(figsize=(12,7))
plt.subplot(211)
for j in range(len(predvidjena_trajektorija_5)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_5[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_5[j], color='black', marker='o', markersize=4)
    else:
        plt.plot(i, predvidjena_trajektorija_5[j], color='red',marker='o',markersize=4)
    i += 1
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Х осе машине (μm)', fontsize=12)
plt.title('Количина тачака која упада у мин/макс појас износи: {}%'.format(round((trajektorija_koord_6/61000)*100,2)) , fontsize=18)
plt.axvline(x = 22100, color = 'b', label = 'axvline - full height')

i = time_steps
plt.subplot(212)
for j in range(len(predvidjena_trajektorija_5)):
    if i%61000 == 0:
        i = time_steps    
    if min_trajektorija[i] <= predvidjena_trajektorija_5[j] <= max_trajektorija[i]:
        plt.plot(i, predvidjena_trajektorija_5[j], color='black', marker='o', markersize=4)
    else:
        plt.plot(i, predvidjena_trajektorija_5[j], color='red',marker='o',markersize=4)
    i += 1
plt.xlabel('време (ms)', fontsize=12)
plt.ylabel('растојање Х осе машине (μm)', fontsize=12)
plt.axvline(x = 22100, color = 'b', label = 'axvline - full height')
plt.show()

# NA TEST SKUPU -tri sigma plotovi

3SIGMA TRAJEKTORIJE

pravimo 3 sigma pojas

In [29]:
tri_sigma_lower = []
tri_sigma_upper = []

for i in range(len(x_command)):
    tri_sigma_lower.append(sr_vrednost[i] - 3*std_dev[i])
    tri_sigma_upper.append(sr_vrednost[i] + 3*std_dev[i])

In [63]:
tri_sigma_lower_trajektorija = []
tri_sigma_upper_trajektorija = []

for i in range(61000):
    tri_sigma_lower_trajektorija.append(x_command[i] + tri_sigma_lower[i])
    tri_sigma_upper_trajektorija.append(x_command[i] + tri_sigma_upper[i])

In [32]:
plt.figure(figsize=(12,7))
plt.plot(tri_sigma_lower[time_steps:], 'blue')
plt.plot(test_predict_MSE[:61000], 'green')
plt.plot(tri_sigma_upper[time_steps:], 'red')
plt.show()

In [65]:
plt.figure(figsize=(12,7))
plt.plot(tri_sigma_lower_trajektorija[time_steps:], 'blue')
plt.plot(predvidjena_trajektorija_0, 'green')
plt.plot(tri_sigma_upper_trajektorija[time_steps:], 'red')
plt.show()

# NA TEST SKUPU - 3 sigma (upadanje u pojas) plotovi

In [25]:
tri_sigma_lower = []
tri_sigma_upper = []

for i in range(len(x_command)):
    tri_sigma_lower.append(sr_vrednost[i] - 3*std_dev[i])
    tri_sigma_upper.append(sr_vrednost[i] + 3*std_dev[i])

In [45]:
for j in range(len(test_file_names)):
    exec(f'trajektorija_odstupanja_{j} = []')
    for i in range(61000 - time_steps):
        exec(f'trajektorija_odstupanja_{j}.append(test_predict_MSE[j*(61000-time_steps) + i])')

plotovanje realizovanih odstupanja

In [47]:
plt.figure(figsize=(11,7))
plt.plot(tri_sigma_lower[time_steps:], color= 'b')
plt.plot(tri_sigma_upper[time_steps:], color = 'r')
for i in range(len(test_file_names)):    
    exec(f'plt.plot(razlika_test_{i}[time_steps::])')
plt.xlabel('време (ms)', fontsize=18)
plt.ylabel('одступање (μm)', fontsize=18)
plt.legend(['три сигма доња граница',"три сигма горња граница", "одступање трајекторије 1","одступање трајекторије 2","одступање трајекторије 3","одступање трајекторије 4","одступање трајекторије 5","одступање трајекторије 6"],loc = 'best')
plt.show()

plotovanje predvidjenih odstupanja

In [46]:
plt.figure(figsize=(11,7))
plt.plot(tri_sigma_lower[time_steps:], color= 'b')
plt.plot(tri_sigma_upper[time_steps:], color = 'r')
for i in range(len(test_file_names)):    
    exec(f'plt.plot(trajektorija_odstupanja_{i})')
plt.xlabel('време (ms)', fontsize=18)
plt.ylabel('одступање (μm)', fontsize=18)
plt.legend(['три сигма доња граница',"три сигма горња граница", "одступање трајекторије 1","одступање трајекторије 2","одступање трајекторије 3","одступање трајекторије 4","одступање трајекторије 5","одступање трајекторије 6"],loc = 'best')
plt.show()

In [75]:
plt.figure(figsize=(11,7))
plt.plot(tri_sigma_lower[time_steps:], color= 'b')
plt.plot(tri_sigma_upper[time_steps:], color = 'r')
plt.plot(trajektorija_odstupanja_0, color = 'g')
plt.xlabel('време (ms)', fontsize=18)
plt.ylabel('одступање (μm)', fontsize=18)
plt.legend(['три сигма доња граница',"три сигма горња граница", "одступање трајекторије 1"],loc = 'best')
plt.show()

In [92]:
trajektorija = 1
i = time_steps
mean_3std = 0
for j in range(len(test_predict_MSE)):
    if i%len(x_command) == 0:
        print("trajektorija ",trajektorija, " sadrzi ", mean_3std , " tacaka", ", procenat tri sigma iznosi: ",mean_3std/61000)
        exec(f"trajektorija_{trajektorija} = mean_3std")
        trajektorija += 1
        mean_3std = 0
        i = time_steps
    if sr_vrednost[i] - 3*std_dev[i] <= test_predict_MSE[j] <= sr_vrednost[i] + 3*std_dev[i]:
            mean_3std += 1
    i += 1
trajektorija_6 = mean_3std
print("trajektorija ",trajektorija, " sadrzi ", mean_3std , " tacaka", ", procenat tri sigma iznosi: ",mean_3std/61000)

trajektorija  1  sadrzi  60044  tacaka , procenat tri sigma iznosi:  0.984327868852459
trajektorija  2  sadrzi  60254  tacaka , procenat tri sigma iznosi:  0.9877704918032787
trajektorija  3  sadrzi  60623  tacaka , procenat tri sigma iznosi:  0.9938196721311475
trajektorija  4  sadrzi  60479  tacaka , procenat tri sigma iznosi:  0.9914590163934426
trajektorija  5  sadrzi  60387  tacaka , procenat tri sigma iznosi:  0.9899508196721312
trajektorija  6  sadrzi  60042  tacaka , procenat tri sigma iznosi:  0.9842950819672132


In [84]:
plt.figure(figsize=(11,7))
for i in range(len(trajektorija_odstupanja_0)):
    if tri_sigma_lower[time_steps + i] <= trajektorija_odstupanja_0[i] <= tri_sigma_upper[time_steps + i]:
        plt.plot(i, trajektorija_odstupanja_0[i], color='black', marker = 'o', markersize = 2)
    else:
        plt.plot(i, trajektorija_odstupanja_0[i], color='red', marker = 'o', markersize = 2)
plt.title('Количина тачака која упада у три сигма појас износи: {}%'.format(round((trajektorija_1/61000)*100,2)) , fontsize=18)
plt.xlabel('време (ms)', fontsize=18)
plt.ylabel('одступање (μm)', fontsize=18)
plt.show()

In [85]:
plt.figure(figsize=(11,7))
for i in range(len(trajektorija_odstupanja_1)):
    if tri_sigma_lower[time_steps + i] <= trajektorija_odstupanja_1[i] <= tri_sigma_upper[time_steps + i]:
        plt.plot(i, trajektorija_odstupanja_1[i], color='black', marker = 'o', markersize = 2)
    else:
        plt.plot(i, trajektorija_odstupanja_1[i], color='red', marker = 'o', markersize = 2)
plt.title('Количина тачака која упада у три сигма појас износи: {}%'.format(round((trajektorija_2/61000)*100,2)) , fontsize=18)
plt.xlabel('време (ms)', fontsize=18)
plt.ylabel('одступање (μm)', fontsize=18)
plt.show()

In [86]:
plt.figure(figsize=(11,7))
for i in range(len(trajektorija_odstupanja_2)):
    if tri_sigma_lower[time_steps + i] <= trajektorija_odstupanja_2[i] <= tri_sigma_upper[time_steps + i]:
        plt.plot(i, trajektorija_odstupanja_2[i], color='black', marker = 'o', markersize = 2)
    else:
        plt.plot(i, trajektorija_odstupanja_2[i], color='red', marker = 'o', markersize = 2)
plt.title('Количина тачака која упада у три сигма појас износи: {}%'.format(round((trajektorija_3/61000)*100,2)) , fontsize=18)
plt.xlabel('време (ms)', fontsize=18)
plt.ylabel('одступање (μm)', fontsize=18)
plt.show()

In [87]:
plt.figure(figsize=(11,7))
for i in range(len(trajektorija_odstupanja_3)):
    if tri_sigma_lower[time_steps + i] <= trajektorija_odstupanja_3[i] <= tri_sigma_upper[time_steps + i]:
        plt.plot(i, trajektorija_odstupanja_3[i], color='black', marker = 'o', markersize = 2)
    else:
        plt.plot(i, trajektorija_odstupanja_3[i], color='red', marker = 'o', markersize = 2)
plt.title('Количина тачака која упада у три сигма појас износи: {}%'.format(round((trajektorija_4/61000)*100,2)) , fontsize=18)
plt.xlabel('време (ms)', fontsize=18)
plt.ylabel('одступање (μm)', fontsize=18)
plt.show()

In [88]:
plt.figure(figsize=(11,7))
for i in range(len(trajektorija_odstupanja_4)):
    if tri_sigma_lower[time_steps + i] <= trajektorija_odstupanja_4[i] <= tri_sigma_upper[time_steps + i]:
        plt.plot(i, trajektorija_odstupanja_4[i], color='black', marker = 'o', markersize = 2)
    else:
        plt.plot(i, trajektorija_odstupanja_4[i], color='red', marker = 'o', markersize = 2)
plt.title('Количина тачака која упада у три сигма појас износи: {}%'.format(round((trajektorija_5/61000)*100,2)) , fontsize=18)
plt.xlabel('време (ms)', fontsize=18)
plt.ylabel('одступање (μm)', fontsize=18)
plt.show()

In [94]:
plt.figure(figsize=(11,7))
for i in range(len(trajektorija_odstupanja_5)):
    if tri_sigma_lower[time_steps + i] <= trajektorija_odstupanja_5[i] <= tri_sigma_upper[time_steps + i]:
        plt.plot(i, trajektorija_odstupanja_5[i], color='black', marker = 'o', markersize = 2)
    else:
        plt.plot(i, trajektorija_odstupanja_5[i], color='red', marker = 'o', markersize = 2)
plt.title('Количина тачака која упада у три сигма појас износи: {}%'.format(round((trajektorija_6/61000)*100,2)) , fontsize=18)
plt.xlabel('време (ms)', fontsize=18)
plt.ylabel('одступање (μm)', fontsize=18)
plt.show()